# Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Reshape, Flatten
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import  fashion_mnist
tf.__version__

# Load Dataset

In [2]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

In [3]:
X_train.shape, Y_train.shape

In [4]:
X_test.shape, Y_test.shape

# Visualize the images

In [5]:
classes = ['T-Shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
width = 10
height = 10

fig, axes = plt.subplots(height, width, figsize=(20, 20))
axes= axes.ravel()
for i in np.arange(0, width*height):
    index = np.random.randint(0, 60000)
    axes[i].imshow(X_train[index], cmap='gray')
    axes[i].set_title(classes[Y_train[index]], fontsize=8)
    axes[i].axis('off')
    
plt.subplots_adjust(hspace=0.4)

# Preprocessing the images

In [6]:
X_train = X_train / 255
X_test = X_test / 255

X_train.shape, X_test.shape

In [7]:
X_train = X_train.reshape((len(X_train), 28, 28, 1))
X_test = X_test.reshape((len(X_test), 28, 28, 1))

X_train.shape, X_test.shape

# Build Convolutional Autoencoder

Output = (Input - Filter + 1) / Stride

In [12]:
autoencoder = Sequential()

#Encoder
autoencoder.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(28, 28, 1)))
autoencoder.add(MaxPooling2D(pool_size=(2, 2)))

autoencoder.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

autoencoder.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same', strides=(2,2)))
autoencoder.add(Flatten())

#Decoder
autoencoder.add(Reshape((4,4,8)))
autoencoder.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D(size=(2,2)))
autoencoder.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D(size=(2,2)))
autoencoder.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
autoencoder.add(UpSampling2D(size=(2,2)))
autoencoder.add(Conv2D(filters=1, kernel_size=(3, 3), activation='sigmoid', padding='same'))

autoencoder.summary()

In [13]:
autoencoder.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
autoencoder.fit(X_train, X_train, epochs=50)

# Encoding And Decoding Test Images

In [15]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('flatten_2').output)

In [16]:
coded_test_images = encoder.predict(X_test)
coded_test_images.shape

In [17]:
decoder_test_images = autoencoder.predict(X_test)
decoder_test_images.shape

In [21]:
n_images = 10
test_images = np.random.randint(0, X_test.shape[0], size=n_images)
plt.figure(figsize=(18, 18))
for i, image_index in enumerate(test_images):
    #original images
    ax = plt.subplot(10,10, i+1)
    plt.imshow(X_test[image_index].reshape(28, 28), cmap='gray')
    plt.xticks(())
    plt.yticks(())
    
    #coded images
    ax = plt.subplot(10, 10, i+1+n_images)
    plt.imshow(coded_test_images[image_index].reshape(16, 8), cmap='gray')
    plt.xticks(())
    plt.yticks(())
    
    #decoded images
    ax = plt.subplot(10, 10, i+1+n_images*2)
    plt.imshow(decoder_test_images[image_index].reshape(28, 28), cmap='gray')
    plt.xticks(())
    plt.yticks(())